In [166]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import random
from selenium.webdriver.support import expected_conditions as EC
import time
from random import randint
# Set the path to your downloaded Edge WebDriver executable

edge_driver_path = r"C:\Users\localadmin\Desktop\myprojects\linkedinsucks\linkedinsucks\code\msedgedriver.exe" 
edge_service = Service(edge_driver_path)
# Initialize the Edge WebDriver
driver = webdriver.Edge(service=edge_service)
driver.implicitly_wait(randint(1,3))

# Initialize the Edge Service

In [167]:
#user details
# define system variables for the url 


linkedin_url = "https://www.linkedin.com"
username = "ganesh_012@outlook.com"
password = "Password123"
role = "Data Scientist"
location = "United States"

In [168]:
def open_linkedin_and_signin():
    driver.get(linkedin_url)
    driver.maximize_window()
    signin_page=driver.find_element("xpath", ".//a[contains(text(), 'Sign in') and @href]")
    signin_page.click()
    username_field = driver.find_element("xpath" , ".//input[@id = 'username']")
    password_field = driver.find_element("xpath" , ".//input[@id = 'password']")
    submit_button = driver.find_element("xpath" , ".//button[@type = 'submit']")
    username_field.send_keys(username)
    password_field.send_keys(password)
    submit_button.click()

In [169]:
# defining job search filters  

def easy_apply():
    easy_apply_button = driver.find_element( "xpath" , "//button[contains(@aria-label ,'Easy Apply filter.')]") 
    easy_apply_button.click()

In [170]:
#opening linkedin 
def get_url(url):
    driver.get(url)
    driver.maximize_window()
    
    
def sign_in(username, password):
    signin_page = driver.find_element("xpath", ".//a[contains(text(), 'Sign in') and @href]")
    signin_page.click()
    username_field = driver.find_element("xpath" , ".//input[@id = 'username']")
    password_field = driver.find_element("xpath" , ".//input[@id = 'password']")
    submit_button = driver.find_element("xpath" , ".//button[@type = 'submit']")
    
    username_field.send_keys(username)
    password_field.send_keys(password)
    submit_button.click()

    
def job_search_and_filters():
    get_url("https://www.linkedin.com/jobs")
    role_input = driver.find_element("xpath" , ".//input[@class = 'jobs-search-box__text-input jobs-search-box__keyboard-text-input jobs-search-global-typeahead__input']")
    role_input.send_keys(role)
    time.sleep(1)
    location_input = driver.find_element("xpath" , ".//input[@aria-label = 'City, state, or zip code' and @autocomplete=  'address-level2']")
    location_input.send_keys(location)
    #submit
    role_input.send_keys(Keys.ENTER)
    time.sleep(2)
    easy_apply()

In [171]:
open_linkedin_and_signin()
job_search_and_filters()

In [172]:
time_dict = {
    "Any time": 0,
    "Past month": 1,
    "Past week": 2,
    "Past 24 hours": 3
}


exp_dict = {
    "Internship": 0,
    "Entry level": 1,
    "Associate": 2,
    "Mid-Senior level": 3,
    "Director": 4,
    "Executive": 5
}

salary_dict = {
    "$40,000+": 0,
    "$60,000+": 1,
    "$80,000+": 2,
    "$100,000+": 3,
    "$120,000+": 4,
    "$140,000+": 5,
    "$160,000+": 6,
    "$180,000+": 7,
    "$200,000+": 8
}

workmode_dict = {
    "On-site": 0,
    "Remote": 1,
    "Hybrid": 2
}


In [202]:
def check_no_matches():
    no_matches  = driver.find_elements('xpath' , ".//h1[@class='t-24 t-black t-normal text-align-center']")
    if no_matches:
        return True
def reset_filters():
    reset_button = driver.find_element('xpath',".//button[@aria-label='Reset applied filters']")
    reset_button.click()



In [205]:
def date_filter( user_date):
    date_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'timePostedRange']  " )
    date_options = date_button.find_elements( "xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    date_button.click()
    label = date_options[time_dict[user_date]].find_element("xpath" , ".//label")
    label.click()
    results = date_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()



def exp_filter( exp_selected):
    exp_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'experience']  " )
    exp_options = exp_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    exp_button.click()
    for exp in exp_selected:
        label = exp_options[exp_dict[exp]].find_element("xpath" , ".//label")
        label.click()
    results = exp_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()


def salary_filter(  base_salary):
    salary_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'salaryBucketV2']  " )
    salary_options = salary_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    salary_button.click()
    label = salary_options[salary_dict[base_salary]].find_element("xpath" , ".//label")
    label.click()
    results = salary_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()


def workmode_filter(  workmode_selected):
    workmode_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'workplaceType']  " )
    hybrid_options = workmode_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    workmode_button.click()
    for mode in workmode_selected:
        label = hybrid_options[workmode_dict[mode]].find_element("xpath" , ".//label")
        label.click()
    results = workmode_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()




def company_filter(companies):

    company_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'company']  ")
    company_button.click()
    input_field = driver.find_element("xpath", ".//input[@aria-label='Add a company']")
    for company in companies:
    
        # input_field = driver.find_element("xpath", ".//input[@aria-label='Add a company']")
        input_field.clear()
        input_field.send_keys(f"{company}")
        time.sleep(0.5)
        input_field.send_keys(Keys.ARROW_DOWN)
        input_field.send_keys(Keys.ENTER)
    results = company_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()



In [174]:
date_params = 'Past month'
exp_param = ['Internship' , 'Entry level']
salary_param = "$80,000+"
workmode_param= ['On-site' , 'Hybrid']
companies = ['blackrock' , 'amazon']


In [206]:
date_filter(date_params)
exp_filter(exp_param)
salary_filter(salary_param)
workmode_filter(workmode_param)
company_filter(companies=companies)